# Sistemas de Recomendación


In [1]:
# 1. Downgrade a NumPy < 2.0 compatible con Surprise
!pip install numpy==1.24.4 --force-reinstall --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 79.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
pymc 5.20.1 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
langchain 0.3.19 requires numpy<2,>=1.26.4; python_version < "3.12", but you have numpy 1.24.4 which is incompatible.


In [ ]:
# 1. Downgrade a NumPy < 2.0 compatible con Surprise
#!pip install numpy==1.24.4 --force-reinstall --no-cache-dir

# 2. Reinstalar Surprise compatible
#!pip install scikit-surprise --no-binary :all: --no-cache-dir

In [24]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469575 sha256=9634e7f65b58cdff2a2ec02f413d762ee389fffae3fda473a6923ddc054888e1
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


# 1. Información de partida: la base de datos

Para el diseño de los sistemas de recomendación se suele disponer de la siguiente información:

* Metadatos o **información** descriptiva **de los contenidos**.
* **Información** sociodemográfica **de los usuarios**.
* **Puntuaciones** o evaluación de los usuarios a los contenidos. Estas puntuaciones pueden ser de manera explícita o implícita:
  - Información explícita: cuando el usuario ha valorado un contenido dando una puntuación concreta (por ejemplo, un valor en una escala de 1 a 5).
  - Información implícita: en algunas aplicaciones es muy complicado pedirle al usuario que realice votaciones de los contenidos; en estos casos, se puede seguir el historial de navegación o de uso del sistema del usuario para saber sus intereses (páginas visitas, contenido previsualizado, número de veces que se escucha una canción...).

La mayor dificultad que nos encontramos para el diseño de estos sistemas es que la mayor parte de esta información no está disponible. De hecho, el objetivo principal de estos sistemas es predecir las puntuaciones que los usuarios darían a contenidos que aún no han valorado.



## **Book-Crossing dataset**

Para esta sesión vamos a trabajar con una base de datos de recomendación de libros: [Book-Crossing](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)

Esta base de datos contiene información de 278.858 usuarios (anonimizados pero con información demográfica) que proporcionan 1.149.780 valoraciones de unos 271.379 libros. Esta información se encuentra estructurada en 3 tablas (o ficheros .csv):

* **Tabla de usuarios**: Contiene la información de los usuarios: un identificador y si están disponibles algunos datos demográficos como la localización y la edad. Como esta información se ha anonimizado, los identificadores de los usuarios son números enteros.  
* **Tabla de libros**: Por cada libro se conoce su identificador (en este caso es el ISBN) y  metadatos adicionales como es el título del libro, el autor, el año de publicación y la editorial.
* **Tabla de puntuaciones**: Contiene la información con las valoraciones que los usuarios han hecho de algunos libros. Las puntuaciones son explícitas e implicitas. Las explicitas vienen expresadas en una escala del 1 al 10 (valores más altos que denotan una mayor interés), y las implícitas están indicadas con un valor 0.

Vamos a cargar la base de datos y analizar el contenido en cada una de estas matrices.

### Matriz de puntuaciones

Vamos a empezar cargando la tabla de puntuaciones y analizando cómo se distribuyen sus valores

In [3]:
import pandas as pd
import csv

ruta = '/content/sample_data/BX-Book-Ratings.csv'

# Leer el archivo ignorando líneas mal formateadas, sin interpretar comillas especiales
rating = pd.read_csv(
    ruta,
    sep=';',
    encoding='latin1',
    quoting=csv.QUOTE_NONE,
    on_bad_lines='skip',
    low_memory=False
)

# Renombrar columnas
rating.columns = ['userID', 'bookID', 'bookRating']

# Limpiar comillas de todas las columnas tipo string
for col in rating.columns:
    if rating[col].dtype == 'object':
        rating[col] = rating[col].str.replace('"', '').str.strip()

# Eliminar duplicados
rating.drop_duplicates(inplace=True)
rating['userID'] = pd.to_numeric(rating['userID'], errors='coerce')
rating['bookRating'] = pd.to_numeric(rating['bookRating'], errors='coerce')

# Ver resultado
rating.head()

,userID,bookID,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
# Check the data table and get some information
print('Número de entradas en la tabla de puntuaciones:',rating.shape[0])
print('Número de usuarios:', len(rating.userID.unique()))
print('Número de libros (items):', len(rating.bookID.unique()))
print('La puntuación máxima es:', rating.bookRating.max())
print('La puntuación mínima es:', rating.bookRating.min())

Número de entradas en la tabla de puntuaciones: 1149780
Número de usuarios: 105283
Número de libros (items): 340553
La puntuación máxima es: 10
La puntuación mínima es: 0


La estructura de esta tabla es una tupla (`userID`, `bookID`, `rating`). La tabla recoge una entrada por cada puntuación emitida. Los usuarios que no han puntuado ningún libro (por ejemplo, nuevos usuarios en el sistema) no tienen ninguna entrada en esta tabla.

### Tabla de contenidos (libros)

Analicemos ahora la información de esta tabla.

In [19]:
books = pd.read_csv(
    '/content/sample_data/BX-Books.csv',
    sep=';',
    encoding='latin1',
    quoting=csv.QUOTE_NONE,
    quotechar='"',    # Indica que las comillas son sólo de texto
    on_bad_lines='skip',  # Salta líneas mal formateadas
    low_memory=False
)
books = books[['"ISBN"', '"Book-Title"', '"Book-Author"', '"Year-Of-Publication"']]

# Limpiar comillas de todas las columnas tipo string
for col in books.columns:
    if books[col].dtype == 'object':
        books[col] = books[col].str.replace('"', '').str.strip()

books.columns = ['bookID', 'title', 'author', 'year']
books['year'] = pd.to_numeric(books['year'], errors='coerce').fillna(0).astype(int)
books.head()

,bookID,title,author,year
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0399135782,The Kitchen God's Wife,Amy Tan,1991


In [20]:
books = books.drop_duplicates(subset=['title', 'author', 'year'])

En este caso cada fila de la tabla está asociada a un libro y tiene la información del mismo (ID, título, autor y año).

### Tabla de usuarios

Analicemos ahora la información de la tabla de usuarios

In [21]:
import pandas as pd
import csv

ruta = '/content/sample_data/BX-Users.csv'

# Leer el archivo ignorando líneas mal formateadas, sin interpretar comillas especiales
users = pd.read_csv(
    ruta,
    sep=';',
    encoding='latin1',
    quoting=csv.QUOTE_NONE,
    on_bad_lines='skip',
    low_memory=False
)

# Renombrar columnas
users.columns = ['userID', 'localication', 'age']

# Limpiar comillas de todas las columnas tipo string
for col in users.columns:
    if users[col].dtype == 'object':
        users[col] = users[col].str.replace('"', '').str.strip()

# Eliminar duplicados
users.drop_duplicates(inplace=True)
users['age'] = pd.to_numeric(users['age'], errors='coerce')

# Ver resultado
users.head()

,userID,localication,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


Al igual que en la tabla de contenido, cada fila de la tabla está asociada a un usuario y tiene la información del mismo. En este caso su ID, localización y año. Vemos que hay campos no disponibles para algunos usuarios y figuran como `NaN`.

### Preprocesado de datos

Antes de empezar a trabajar con esta base de datos, vamos a filtrar algunos de sus campos en base a dos criterios:

1. Vamos a eliminar las puntuaciones implicitas. La mayoría de los modelos que vamos a ver solo trabajan con puntuaciones explicitas, así que vamos a eliminar las implicitas para facilitar el diseño de los modelos.

2. Para reducir el tamaño del conjunto de datos, y evitar encontrarnos con largas ejecuciones, vamos a filtrar los libros con pocas puntuaciones y los usuarios que han puntuado pocos libros. De hecho, estos usuarios/libros no nos iban a dar buenos resultados, así que al eliminarlos faciliamos el análisis de los métodos que vamos a ver.




In [22]:
print('El tamaño de la matriz de puntuaciones original es:\t{}'.format(rating.shape))


# 1. Remove implicit ratings

rating = rating[rating.bookRating>0]

# 2. Remove users and books with a low number of ratings

min_book_ratings = 50
filter_books = rating.bookID.value_counts() > min_book_ratings
filter_books = filter_books[filter_books].index.tolist()

min_user_ratings = 50
filter_users = rating.userID.value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

rating = rating[(rating.bookID.isin(filter_books)) & (rating.userID.isin(filter_users))]
print('El tamaño de la nueva matriz de puntuaciones es:\t{}'.format(rating.shape))

users = users[(users.userID.isin(filter_users))]
print('El tamaño de la nueva matriz de usuarios es:\t{}'.format(users.shape))

books = books[(books.bookID.isin(filter_books))]
print('El tamaño de la nueva matriz de libros es:\t{}'.format(books.shape))

El tamaño de la matriz de puntuaciones original es:	(1149780, 3)
El tamaño de la nueva matriz de puntuaciones es:	(13716, 3)
El tamaño de la nueva matriz de usuarios es:	(0, 3)
El tamaño de la nueva matriz de libros es:	(486, 4)


# 2. Filtrado colaborativo

Los sistemas de filtrado colaborativo buscan similitudes entre usuarios o contenidos en base a las puntuaciones que los usuarios han dado a los contenidos y no haciendo uso de la información demográfica de los usuarios o de los metadatos asociados a los contenidos.

Estos sistemas se dividen en dos tipos:

1. **Filtrado basado en el usuario**: estos sistemas recomiendan a un usuario productos que han gustado a usuarios similares. Por ejemplo, digamos que Alicia y María tienen un gustos similares en los libros (es decir, en gran medida les gustan y no les gustan los mismos libros). Digamos que se ha lanzado un nuevo libro al mercado y que Alicia lo ha leído y le ha gustado. Por lo tanto, es muy probable que a María también le guste, y por lo tanto, el sistema recomiendaría este libro a María.

2. **Filtrado basado en contenido**: estos sistemas se parecen al sistema que acabamos de diseñar (basados en contenido), salvo por el hecho que las similutes entre los contenidos (o items) se calculan en base a las puntuaciones que les han dado los usuarios. Por ejemplo, si Alicia, María y Ana han dado 5 estrellas a los libros de Harry Potter y Crepúsculo, el sistema identifica los artículos como similares. Por lo tanto, si alguien compra uno de los libros de Harry Potter, el sistema también le recomienda la saga de Crepúsculo.

Para diseñar estos sistemas solo necesitamos la matriz de puntuaciones (`rating`).

Y ahora que conozco algunos de los modelos de filtrado colaborativo, ¿cómo los aplico a mi problema de recomendación de libros?

Podríamos implementarnos estos modelos en Python y diseñar nuestras propias funciones, pero aquí vamos a hacer uso de una de las librerias en Python para Sistemas de Recomendación más utilizadas: [Surprise](http://surpriselib.com/)

# Item-based filtering (filtrado colaborativo basado en memoria entre ítems)
SURPRISE

La libreria [SurPRISE](http://surpriselib.com/) (Simple Python RecommendatIon System Engine) nos facilita el diseño de los sistemas de recomendación (en especial los sistemas de filtrado colaborativo) incluyendo herramientas para:

* Manejar los datos (en especial las matrices de puntuaciones). Los usuarios pueden usar conjuntos de datos incorporados (Movielens, Jester) o bases de datos propias.
* Entrenar multiples algoritmos de filtrado colaborativo: los que acabamos de ver basados en vencidad entre usuarios o entre contenido, así como otros métodos de filtrado colaborativo que veremos más adelante.
* Facilitar la evaluación, análisis y comparación de las prestaciones de diferentes métodos.
* Realizar el ajuste de parámetros. Para ello incluye procedimientos de validación cruzada.

Todos estos métodos o herramientas tienen una sintaxis muy similar a las de sklearn, así que resulta muy fácil su uso.

Vayamos viendo cómo trabajar con esta libreria sobre nuestro ejemplo. Para ello empecemos instalando Surprise en Colab.

## 5.1. Carga  y preprocesado de datos en Surprise

Para cargar un conjunto de datos desde el dataframe a *Surprise*, podemos usar el método `load_from_df()`. Para ello, debemos partir de un dataframe con tres columnas (en este orden):
* ids de usuario,
* ids de los contenidos,
* puntuaciones asignadas.

Además, esta función utiliza un objeto `Reader` donde se debe especificar el parámetro `rating_scale` (rango de valores de las puntuaciones, en nuestro caso de 0 a 9).




In [25]:
from surprise import Reader, Dataset

reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(rating[['userID', 'bookID', 'bookRating']], reader)

Veamos el antes y después...

In [26]:
rating.head()

,userID,bookID,bookRating
1456,277427,002542730X,10
1474,277427,0061009059,9
1522,277427,0316776963,8
1543,277427,0345413903,10
1581,277427,0385486804,9


In [27]:
print(type(data))
data.raw_ratings[:10]

<class 'surprise.dataset.DatasetAutoFolds'>


[(277427, '002542730X', 10.0, None),
 (277427, '0061009059', 9.0, None),
 (277427, '0316776963', 8.0, None),
 (277427, '0345413903', 10.0, None),
 (277427, '0385486804', 9.0, None),
 (277427, '0385504209', 8.0, None),
 (277427, '0399501487', 9.0, None),
 (277427, '0440224764', 7.0, None),
 (277427, '0452284449', 6.0, None),
 (277427, '0552137030', 8.0, None)]

Para entrenar nuestro modelo, al igual que cuando trabajamos con modelos de clasificación/regresión, es adecuado dividir el conjunto de entrenamiento en particiones de entrenamiento y test. Para ello podemos usar la función `train_test_split()`.

In [28]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

Viendo el código así, todo parece muy sencillo y parecido a lo que hacemos en sklearn. Pero **cuidado**, Surprise maneja diferentes tipos de datos, y al estar haciendo estas conversiones genera tipos de datos diferentes y utiliza índices internos para los usuarios y los items...

In [29]:
print(type(trainset))
# Esta objeto permite obtener las tuplas (id_u, id_i, rating) con el método .all_ratings()
print('Valores de los ratings')
print(list(trainset.all_ratings())[:10])
# Podemos usar las funciones .to_raw_iid(), .to_raw_uid para convertir los ids internos a los originales.
# O las funciones .to_inner_iid()   y  .to_inner_uid() para hacer la conversión inversa.

<class 'surprise.trainset.Trainset'>
Valores de los ratings
[(0, 0, 9.0), (0, 63, 8.0), (0, 398, 10.0), (0, 177, 9.0), (0, 26, 10.0), (0, 135, 8.0), (0, 167, 8.0), (0, 217, 6.0), (0, 146, 9.0), (0, 236, 7.0)]


In [30]:
print(type(testset))
testset[:10]

<class 'list'>


[(75591, '0802130208', 10.0),
 (221445, '0446601241', 8.0),
 (126604, '0767902521', 7.0),
 (271622, '0385512104', 10.0),
 (10560, '0439139600', 9.0),
 (30711, '0345391802', 7.0),
 (2977, '0316569321', 7.0),
 (7346, '0440221471', 9.0),
 (69971, '038542017X', 7.0),
 (37538, '0971880107', 7.0)]

Como veremos, estos objetos, `trainset` y `testset`, son los que usan los métodos .fit() y .predict() de los diferentes modelos. Si queremos entrenar un modelo con el conjunto de datos original, habría que convertir el tipo de datos `surprise.dataset.DatasetAutoFolds` a `surprise.trainset.Trainset` o `list`, según quisieramos entrenar o predecir. Esto podemos hacerlo con las funciones `build_full_trainset()` y `build_testset()` o `build_anti_testset()` tal y como mostramos en el siguiente ejemplo:

In [31]:
# Para comprobar vemos el numero de ratings en data
print('Datos originales')
print(len(data.raw_ratings))

# Generamos un conjunto de ratings con los datos de entrenamiento
trainset_all = data.build_full_trainset()
print('Datos conjunto de entrenamiento')
print(type(trainset_all))
print(trainset_all.n_ratings)

# Generamos un conjunto de test con los datos de entrenamiento (para tener prestaciones en train)
testset_all = trainset_all.build_testset()
print('Datos conjunto de test (con los datos de entrenamiento')
print(type(testset_all))
print(len(testset_all))

# Generamos un conjunto de test con los datos que no están en entrenamiento (todas las conbinaciones de usuarios e items sin rating)
#testset_all_anti = trainset_all.build_anti_testset()
#print('Datos conjunto de test (los datos que no están en entrenamiento)')
#print(type(testset_all_anti))
#print(len(testset_all_anti))


Datos originales
13716
Datos conjunto de entrenamiento
<class 'surprise.trainset.Trainset'>
13716
Datos conjunto de test (con los datos de entrenamiento
<class 'list'>
13716


## 5.2 Entrenar un modelo

Al igual que sklearn, para entrenar un modelo solo tenemos que definir el modelo con su constructor y luego usar el método `.fit()` para entrenarlo. Para entrenar un modelo basado en vecinos, Surprise incluye los siguientes métodos:

* [`KNNBasic`](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBasic): es el algoritmo básico de filtrado colaborativo . Permite realizar el filtrado colaborativo basado en usuarios o contenido según definamos el parámetro `user_based` de [`sim_options`](https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration).

* [`KNNWithMeans`](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans): es el mismo algoritmo que el anterior pero con corrección de medias.

* [`KNNWithZScore`](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithZScore):  es el mismo algoritmo que los anteriores pero con corrección de medias y desviación estandar.  

Al definir estos modelos, Surprise nos deja seleccionar dos parámetros que afectan al vecindario que se usa al hacer las predicciones. Si recordamos que las predicciones vienen dadas por:

$$ \hat{r}_{u,i} = \frac{\sum_{v \in N_u^K} {\rm sim}(u,v) r_{v,i} }{\sum_{v \in N_u^K} {\rm sim}(u,v)} $$

donde $N_u^K$ son los $K$ vecinos más similares al usuario $u$, Surprise nos deja controlar este vecindario con estos parámetros:
* `k`: número de vecinos (usuarios/items) que se consideran al hacer la predicción (tamaño del conjunto $N_u^K$). Si un usuario tiene más vecinos, solo se usan los `k` más parecidos. Y si tiene menos, se consideran los que tienen a no ser que sean menos que `k_min`.
* `k_min`: número mínimo de vecinos que debe tener un usuario/item para realizar una predicción. Para los usuarios/items que no tengan tantos vecinos directamente se hace una predicción basada en sus medias, es decir, en $\bar{r}_{u}$ o  $\bar{r}_{i}$.

Además, dentro de la clase  [`sim_options`](https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration), no solo nos permite seleccionar un modelo basado en usuarios y en contenido, sino que nos permite definir varias métricas, importadas del módulo de [`similitudes`](https://surprise.readthedocs.io/en/stable/similarities.html#module-surprise.similarities), para encontrar los usuarios o items más parecidos a uno dado. En concreto, incluye 4 métricas:

* `cosine`: 	Calcula la similitud del coseno entre todos los pares de usuarios (o items). Por ejemplo, entre los usuarios $u$ y $v$ está medida sería:
$${\rm sim}(u,v) = \frac{\sum_{i \in I_{u,v}}  r_{u,i}  r_{v,i}  }{\sqrt{\sum_{i \in I_{u,v}} r_{u,i}^2  \sum_{i \in I_{u,v}}r_{v,i}^2}}$$
donde $I_{u,v}$ es el conjunto de items que ambos usuarios han puntuado.

* `msd`: 	Calcula la similitud como la inversa de la distancia cuadrática media entre todos los pares de usuarios (o items). En este caso la similitud entre los usuarios $u$ y $v$ viene dada por:
$${\rm sim}(u,v) = \frac{1}{1 + \sum_{i \in I_{u,v}}  \left(r_{u,i} - r_{v,i} \right)^2}$$
donde se incluye un término $+1$ en el denominador para evitar divisiones entre $0$.

* `pearson`: 	Calcula el coeficiente de correlación de Pearson entre todos los pares de usuarios (o ítems), dado por:
$${\rm sim}(u,v) = \frac{\sum_{i \in I_{u,v}}  \left(r_{u,i} - \bar{r_{u}}\right)  \left(r_{v,i}- \bar{r_{v}} \right) }{\sqrt{\sum_{i \in I_{u,v}} \left(r_{u,i} - \bar{r_{u}}\right)^2  \sum_{i \in I_{u,v}} \left( r_{v,i} - \bar{r_{v}}\right)^2}}$$
donde $\bar{r_{u}}$  y $\bar{r_{v}}$ son los valores medios de las puntuaciones de los usuarios $u$ y $v$.

* `pearson_baseline`: Es la misma medida que antes salvo porque en vez de restar las medias a las puntuaciones les resta unos coefientes de media que debe aprender (ya veremos más adelante cómo se aprenden) y porque incluye una corrección (*shrinkage*) para evitar efectos de sobreajuste cuando hay pocos elementos comunes entre los usuarios $u$ y $v$. Con estos cambios esta medida viene dada por:
$${\rm sim}(u,v) =\frac{\mid I_{u,v} \mid -1}{\mid I_{u,v} \mid -1+Sh} \frac{\sum_{i \in I_{u,v}}  \left(r_{u,i} - {b_{u}}\right)  \left(r_{v,i}- {b_{v}} \right) }{\sqrt{\sum_{i \in I_{u,v}} \left(r_{u,i} - {b_{u}}\right)^2  \sum_{i \in I_{u,v}} \left( r_{v,i} - {b_{v}}\right)^2}}$$
donde ${b_{u}}$  y ${b_{v}}$ son los coeficientes de medias de los usuarios $u$ y $v$ (que deberemos aprender) y $Sh$ es el coeficiente *shrinkage* que es un parametro del modelo a definir.

Una vez sabemos los modelos que incluye Surprise,
veamos cómo definir y entrenar uno de estos modelos. Empecemos por el modelo más sencillo, el `KNNBasic` con sus parámetros por defecto.

In [32]:
from surprise.prediction_algorithms.knns import KNNBasic

KNNalgo = KNNBasic()
KNNalgo.fit(trainset)

# Check the model parameters
print(KNNalgo.k)
print(KNNalgo.min_k)
print(KNNalgo.sim_options)

Computing the msd similarity matrix...
Done computing similarity matrix.
40
1
{'user_based': True}


Si queremos podemos modificar estos parámetros en el constructor. Por ejemplo, podemos poner un vecindario de 60 y un `k_min` de 10.

In [33]:
KNNalgo = KNNBasic(k=60, min_k=10)
KNNalgo.fit(trainset)

# Check the model parameters
print(KNNalgo.k)
print(KNNalgo.min_k)
print(KNNalgo.sim_options)

Computing the msd similarity matrix...
Done computing similarity matrix.
60
10
{'user_based': True}


## 5.3 Obtener predicciones

Un vez entrenado el modelo, en Surprise podemos usar el método [`.predict()`](https://surprise.readthedocs.io/en/stable/predictions_module.html#surprise.prediction_algorithms.predictions.Prediction) para obtener la predicción de un item concreto para un usuario dado. O si queremos obtener las predicciones para un conjunto de test (varios usuarios e items a la vez) podemos usar el método `.test()`.


Si activamos el `verbose` del método `predict`, éste saca por pantalla información sobre la predicción. Entre esta información se muestra el valor real (si se lo indicamos al llamarlo), el valor predicho y un flag `PredictionImpossible` que nos indica si era posible realizar la predicción. Por ejemplo, en los métodos KNN si no hay suficientes vecinos (`min_k') no se puede realizar la predicción y este método activa esta excepción y como predicción se devuelve el valor medio de las puntuaciones.


#### **Ejercicio 7**:
Estima la puntuación de los 4 primeros elementos en el conjunto de test

#### Solución

In [34]:
# Check the four first elements in the test dataset
testset[:4]

[(75591, '0802130208', 10.0),
 (221445, '0446601241', 8.0),
 (126604, '0767902521', 7.0),
 (271622, '0385512104', 10.0)]

In [35]:
uid = 14521  # user id (integer)
iid = '0553268880'  # book id (string)

# get a prediction for specific users and items.
pred = KNNalgo.predict(uid, iid, verbose=True)

user: 14521      item: 0553268880 r_ui = None   est = 7.90   {'actual_k': 11, 'was_impossible': False}


In [36]:
for sample in range(4):

  uid = testset[sample][0]  # user id (integer)
  iid = testset[sample][1]  # book id (string)
  rui = testset[sample][2]  # rating (we include it just for additional information)
  # get a prediction for specific users and items.
  pred = KNNalgo.predict(uid, iid, rui, verbose=True)

user: 75591      item: 0802130208 r_ui = 10.00   est = 8.04   {'was_impossible': True, 'reason': 'Not enough neighbors.'}
user: 221445     item: 0446601241 r_ui = 8.00   est = 7.78   {'actual_k': 10, 'was_impossible': False}
user: 126604     item: 0767902521 r_ui = 7.00   est = 8.04   {'was_impossible': True, 'reason': 'Not enough neighbors.'}
user: 271622     item: 0385512104 r_ui = 10.00   est = 8.04   {'was_impossible': True, 'reason': 'Not enough neighbors.'}


#### **Ejercicio 8**:
Estima las puntuaciones de todo el conjunto de test

#### Solución


In [37]:
# Get predictions
predictions = KNNalgo.test(testset)
# Check the output for the first values
print(predictions[:10])

[Prediction(uid=75591, iid='0802130208', r_ui=10.0, est=8.03742587732089, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}), Prediction(uid=221445, iid='0446601241', r_ui=8.0, est=7.77951526524013, details={'actual_k': 10, 'was_impossible': False}), Prediction(uid=126604, iid='0767902521', r_ui=7.0, est=8.03742587732089, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}), Prediction(uid=271622, iid='0385512104', r_ui=10.0, est=8.03742587732089, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}), Prediction(uid=10560, iid='0439139600', r_ui=9.0, est=9.631992963678707, details={'actual_k': 16, 'was_impossible': False}), Prediction(uid=30711, iid='0345391802', r_ui=7.0, est=8.03742587732089, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}), Prediction(uid=2977, iid='0316569321', r_ui=7.0, est=8.03742587732089, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}), Prediction(uid=7346, iid='0440221471

## 5.4. Evaluar el modelo

Una vez tenemos las predicciones, podemos usar algunas de las siguientes medidas o métricas en el módulo [accuracy](https://surprise.readthedocs.io/en/stable/accuracy.html) para evaluar prestaciones. En concreto hay 4 medidas para evaluar el error:

* `rmse`: Calcula la raíz del error cuadrático medio (RMSE, *Root Mean Squared Error*):
$$ RMSE = \sqrt{\frac{1}{N} \sum_{i=1}^N (p_i -r_i)^2}$$
* `mse`: Calcula el error cuadrático medio (MSE, *Mean Squared Error*):
$$ MSE = \frac{1}{N} \sum_{i=1}^N (p_i -r_i)^2$$
* `mae`: Calcula el error absoluto medio (MAE, *Mean Absolute Error*):
$$MAE  = \frac{1}{N} \sum_{i=1}^N |p_i -r_i|$$
* `fcp`: Calcula 	el cociente de pares concordantes (FCP, *Fraction of Concordant Pairs*). Esta medida está orientada a evaluar ranking y no valores exactos de la puntuación. Para ello, miden el número de parejas concordantes por usuario contando el número de pares:
$$n^u_c=\mid \{(i,j), ~ \hat{r}_{u,i}>\hat{r}_{u,j} ~~\rm{y} ~~~{r}_{u,i}>{r}_{u,j}\}\mid$$
y de manera equivalente evalúan el número de parejas discordantes $n^u_d$. A continuación, promediando estos valores sobre todos los usuarios, $n_c = \sum_u n^u_c$ y $n_d = \sum_u n^u_d$, se define el FCP como:
$$ FCP = \frac{n_c}{n_c+n_d}$$

Para utiliar estas métricas solo tenemos que llamar a la función correspondiente dentro del módulo `accuracy` dándole el conjunto de predicciones que hemos estimado (tenga en cuenta que en estas prediciones también va incluido el valor real para que se pueda calcular el error).


In [38]:
from surprise import accuracy

# Compute performance
RMSE = accuracy.rmse(predictions)

MSE = accuracy.mse(predictions)

MAE = accuracy.mae(predictions)

FCP = accuracy.fcp(predictions)

RMSE: 1.7293
MSE: 2.9906
MAE:  1.3139
FCP:  0.6079


## 5.5 Validación de parámetros

Surprise proporciona varias herramientas para ejecutar procedimientos de validación cruzada y buscar los mejores parámetros para un algoritmo dado. Las herramientas son fáciles de usar, ya que siguen el formato de sklearn.

Por un lado, tiene funciones para generar particiones de entrenamiento y test (como la que hemos usado anteriormente) o para generar diferentes particiones de validación dentro del conjunto de entrenamiento (en este [link](https://surprise.readthedocs.io/en/stable/model_selection.html#module-surprise.model_selection.split) puede encontrar un detalle de todas ellas).

Por otro lado, al igual que sklearn tiene el módulo `GridSearch`, Surprise incluye la función `GridSearchCV` que nos permite definir un modelo, los parámetros a explorar y se encarga de hacer toda la magia por nosotros.

Para ver cómo funciona, vamos a entrenar un modelo de recomendación por vecindad básico (`KNNBasic`) basado en contenido (`'user_based': False `) y seleccionar por CV el número de vecinos (`k`) y el número mínimo de vecinos (`min_k`).

Nota: La función Gridsearch está diseñada para trabajar con datos del tipo `surprise.dataset.DatasetAutoFolds`, así que de momento nos vamos olvidar de las particiones de entrenamiento/test que hemos hecho y validaremos el modelo usando todos los datos en `data`. Luego veremos cómo podemos utilizar particiones de entrenamiento y test.

A partir de aquí aplicamos Grid Search (búsqueda en grilla) para encontrar
 los mejores hiperparámetros del modelo KNNBasic sobre un dataset filtrado.
 - Filtramos previamente usuarios e ítems con pocas interacciones para evitar errores como divisiones por cero en el cálculo de similitudes.
 - Usamos 'msd' (Mean Squared Difference) como métrica de similitud, que es más robusta que cosine en datos dispersos.
 - Limitamos el tamaño de la grilla para evitar excesivo consumo de RAM.



In [39]:
from surprise import Dataset, Reader
from surprise.model_selection.search import GridSearchCV
from surprise.prediction_algorithms.knns import KNNBasic

# Filtrar usuarios e ítems con al menos 5 ratings
min_ratings = 5
user_counts = rating['userID'].value_counts()
item_counts = rating['bookID'].value_counts()

filtered_rating = rating[
    rating['userID'].isin(user_counts[user_counts >= min_ratings].index) &
    rating['bookID'].isin(item_counts[item_counts >= min_ratings].index)
]

# Generar dataset filtrado para Surprise
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(filtered_rating[['userID', 'bookID', 'bookRating']], reader)

# Grid Search con parámetros razonables
param_grid = {
    'k': [10, 20],
    'min_k': [1, 3],
    'sim_options': {
        'name': ['msd'],
        'user_based': [False]
    }
}
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mse'], cv=3)

# Intentar ajuste y capturar errores
try:
    gs.fit(data)
except ZeroDivisionError as e:
    print("Se produjo un error de división por cero al ajustar el modelo:", e)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


Podemos analizar los resultados del proceso de CV analizando el contenido de `cv_results`

In [40]:
results_df = pd.DataFrame.from_dict(gs.cv_results)
results_df.head()

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mse,split1_test_mse,split2_test_mse,mean_test_mse,std_test_mse,rank_test_mse,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_k,param_min_k,param_sim_options
0,1.698801,1.717174,1.664468,1.693481,0.021844,4,2.885925,2.948685,2.770452,2.868354,0.073816,4,0.007066,0.001387,0.077950,0.003896,"{'k': 10, 'min_k': 1, 'sim_options': {'name': ...",10,1,"{'name': 'msd', 'user_based': False}"
1,1.663842,1.691292,1.628776,1.661303,0.025585,2,2.768371,2.860468,2.652911,2.760583,0.084914,2,0.005286,0.000165,0.070390,0.000594,"{'k': 10, 'min_k': 3, 'sim_options': {'name': ...",10,3,"{'name': 'msd', 'user_based': False}"
2,1.693002,1.713625,1.663147,1.689925,0.020722,3,2.866256,2.936509,2.766058,2.856274,0.069943,3,0.005797,0.000381,0.070595,0.001182,"{'k': 20, 'min_k': 1, 'sim_options': {'name': ...",20,1,"{'name': 'msd', 'user_based': False}"
3,1.657921,1.687688,1.627426,1.657679,0.024603,1,2.748702,2.848292,2.648516,2.748504,0.081558,1,0.005167,0.000117,0.067264,0.003497,"{'k': 20, 'min_k': 3, 'sim_options': {'name': ...",20,3,"{'name': 'msd', 'user_based': False}"


O directamente podemos ver el mejor resultado y los parámetros óptimos en `best_score` y `best_params`.

In [41]:
# Mejor RMSE
print(gs.best_score['rmse'])

# Parametros óptimos
print(gs.best_params['rmse'])

1.6576786056306692
{'k': 20, 'min_k': 3, 'sim_options': {'name': 'msd', 'user_based': False}}


Este es el mejor RMSE (Root Mean Squared Error) obtenido en validación cruzada.

Significa que, en promedio, las predicciones del modelo tienen un error cuadrático medio de aproximadamente 1.65 unidades sobre la escala de ratings (en este caso, entre 1 y 10).

Cuanto más bajo es el RMSE, mejor es la capacidad del modelo de predecir ratings cercanos al valor real.

Parametros:

* El modelo usa los 20 vecinos más similares para predecir la puntuación.

* Necesita al menos 3 vecinos con puntuación válida para realizar una predicción.

* Se usa Mean Squared Difference como métrica de similitud. Es más robusta que cosine con datos dispersos.

* Indica que la similitud se calcula entre ítems (item-item), no entre usuarios.


**¿Es bueno un RMSE de 3.73?**

Depende del contexto y la escala:

Si tus ratings van de 1 a 10, un RMSE de 3.7 indica que las predicciones están, en promedio, a unos 3.7 puntos del valor real.

No es excelente, pero tampoco catastrófico — refleja que el dataset es grande, disperso o ruidoso.

**¿Cómo mejorarlo?**

Probar modelos basados en modelos latentes como SVD o SVDpp.

Incluir más datos por usuario/ítem (filtro más agresivo).

Probar técnicas híbridas con features de contenido si están disponibles.

## 5.7 Limitaciones de los modelos basados en vecindad

* ¿Cómo podemos hacer recomendaciones a nuevos usuarios del sistema o a nuevos contenidos? Esto es lo que se conoce como **cold-start-problem** y su solución pasa por crear sistemas híbridos que combinen sistemas de filtrado colaborativo con sistemas basados en popularidad, usuario o contenido. Por ejemplo:
  * El problema de los nuevos usuarios puede ser aliviado con un sistema híbrido que incorpore información demográfica de los usuarios para encontrar usuarios parecidos. O pedirle a los nuevos usuarios que puntúen un número mínimo de objetos para que el sistema pueda empezar a funcionar.
  * El problema de los nuevos contenidos también se puede aliviar con sistemas híbridos que, en este caso, incorporen información de metadatos de contenidos. O pidiendo a los usuarios que puntúen a los nuevos artículos (y darles alguna recompensa por ello).

* Y aunque los usuarios y contenidos no sean nuevos, en ocasiones las puntuaciones que tienen son demasiado pocas para hacer buenas estimaciones y directamente las puntuaciones se estiman por medias. En Surprise se puede incluir un umbral sobre el número mínimo de puntuaciones comunes entre dos usuarios o items para que se calcule su similitud.

* Coste computacional de estos modelos. Tal y como hemos visto que funcionan estos modelos, para estimar la puntuación de un usuario $u$ por un item $i$, necesitamos calcular los usuarios parecidos al usuario $u$ (en un modelo basado en usuario) o parecidos al item $i$ (en el modelo basado en contenido). Este cálculo conlleva un gran coste computacional ya que requiere calcular las distancias entre todos los usuarios o items (accediendo a todas las puntuaciones de todos los usuarios o items) para poder hacer una estimación; estos modelos son los que se conocen como **basados en memoria** y solo pueden usarse (desde un punto de vista práctico) si la base de datos no es muy grande. Para solventar esta limitación, en ocasiones se plantea precalcular estas distancias (saber qué usuarios o items son parecidos a uno dado), generando un sistema **basado en modelo**, y así solo hay que promediar puntuaciones para estimar una nueva. Este esquema suele funcionar muy bien en modelos de vecindad basados en contenido ya que las similitudes entre items no suelen variar con el tiempo; de hecho, éste es el esquema que usa Amazon en su sistema de recomendación y con él es capaz de hacer recomendaciones sobre millones de productos de manera muy eficiente.



# Referencias

Charu C. Aggarwal. Recommender Systems: The Textbook. 2016.

Surprise ([artículos de referencia](https://surprise.readthedocs.io/en/stable/notation_standards.html#))
